<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/03_Analisis_Fase_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
================================================================================
NOTEBOOK 3 - FASE 1: INTEGRACIÓN DE DATOS (VERSIÓN COMPLETA)
================================================================================
Trabajo Fin de Máster: Evaluación Comparativa de Técnicas de Segmentación
en Fotografía de Retrato

Autor: Jesús L.
Universidad: Universidad Oberta de Cataluña (UOC)
Fecha: Noviembre 2025

DESCRIPCIÓN:
Fase 1 del análisis comparativo de modelos de segmentación. Integra datos de
características fotográficas (Notebook 00), máscaras de modelos de segmentación
(Notebooks 01-04), metadata de ejecución y ground truth manual anotado en CVAT
en una estructura unificada para procesamiento posterior.

COMPONENTES PRINCIPALES:
1. Lector de características fotográficas con manejo defensivo de EXIF
2. Lector de ground truth desde anotaciones CVAT
3. Escaneo dinámico y recursivo de configuraciones de modelos
4. Carga de máscaras NPZ con estructuras heterogéneas
5. Carga de metadata de ejecución (tiempos, scores, configuraciones)
6. Consolidación en estructura DatosFotografia por imagen
7. Validación de integridad de datos con control de errores críticos

MEJORAS EN ESTA VERSIÓN:
- Logging sin duplicación (Google Colab compatible)
- Búsqueda recursiva de configuraciones de modelos
- Soporte para JSONs en subdirectorios
- Carga de metadata de ejecución de modelos
- Búsqueda flexible de máscaras NPZ en múltiples ubicaciones
- Unificación de metadata de diferentes modelos

CRITERIOS DE ERROR:
- ERROR CRÍTICO (detiene ejecución): Falta Ground Truth, falta características
- WARNING (continúa): Falta máscara de un modelo específico, campos opcionales

Referencias:
- Gonzalez, R.C. & Woods, R.E. (2018). Digital Image Processing (4th ed.)
- Lin, T.Y., et al. (2014). Microsoft COCO: Common Objects in Context
================================================================================
"""

'\n================================================================================\nNOTEBOOK 3 - FASE 1: INTEGRACIÓN DE DATOS (VERSIÓN COMPLETA)\n================================================================================\nTrabajo Fin de Máster: Evaluación Comparativa de Técnicas de Segmentación\nen Fotografía de Retrato\n\nAutor: Jesús L.\nUniversidad: Universidad Oberta de Cataluña (UOC)\nFecha: Noviembre 2025\n\nDESCRIPCIÓN:\nFase 1 del análisis comparativo de modelos de segmentación. Integra datos de \ncaracterísticas fotográficas (Notebook 00), máscaras de modelos de segmentación \n(Notebooks 01-04), metadata de ejecución y ground truth manual anotado en CVAT \nen una estructura unificada para procesamiento posterior.\n\nCOMPONENTES PRINCIPALES:\n1. Lector de características fotográficas con manejo defensivo de EXIF\n2. Lector de ground truth desde anotaciones CVAT\n3. Escaneo dinámico y recursivo de configuraciones de modelos\n4. Carga de máscaras NPZ con estructuras hetero

In [ ]:
import os
import gc
import json
import logging
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple
from dataclasses import dataclass, field, asdict

import numpy as np
from PIL import Image

In [ ]:
# Verificar si estamos en Colab
try:
    import google.colab
    IN_COLAB = True
    print("\nEntorno: Google Colab detectado")
except ImportError:
    IN_COLAB = False
    print("\nEntorno: Ejecución local (no Colab)")

# Montar Google Drive si estamos en Colab
if IN_COLAB:
    print("\nMontando Google Drive...")
    print("-"*80)

    from google.colab import drive

    try:
        drive.mount('/content/drive', force_remount=False)
        print("Google Drive montado correctamente en: /content/drive")

        # Verificar que el directorio TFM existe
        ruta_tfm_check = Path("/content/drive/MyDrive/TFM")
        if ruta_tfm_check.exists():
            print(f"Directorio TFM encontrado: {ruta_tfm_check}")
        else:
            print(f"\nADVERTENCIA: No se encontró directorio TFM en {ruta_tfm_check}")
            print("Verifica que la estructura del proyecto esté correctamente ubicada")

    except Exception as e:
        print(f"ERROR montando Google Drive: {e}")
        print("Verifica permisos y conexión a Drive")
        raise
else:
    print("\nEjecución local - saltando montaje de Drive")
    print("-"*80)

print("\n")


Entorno: Google Colab detectado

Montando Google Drive...
--------------------------------------------------------------------------------
Mounted at /content/drive
Google Drive montado correctamente en: /content/drive
Directorio TFM encontrado: /content/drive/MyDrive/TFM




In [ ]:
# =============================================================================
# CONFIGURACIÓN DE LOGGING
# =============================================================================

def configurar_logging() -> logging.Logger:
    """
    Configura sistema de logging para la Fase 1.

    Elimina handlers previos para evitar duplicación de mensajes
    en Google Colab cuando se re-ejecutan las celdas.

    Returns:
        Logger configurado con formato académico
    """
    logger = logging.getLogger('Fase1_Integracion')

    # CRÍTICO: Eliminar todos los handlers previos para evitar duplicación
    # Esto es necesario en Google Colab donde las celdas se re-ejecutan
    if logger.hasHandlers():
        logger.handlers.clear()

    logger.setLevel(logging.INFO)

    # Handler para consola
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)

    # Formato académico sin emoticones
    formatter = logging.Formatter(
        '[%(asctime)s] %(levelname)s - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )
    console_handler.setFormatter(formatter)

    logger.addHandler(console_handler)

    # Evitar que los mensajes se propaguen al logger raíz
    logger.propagate = False

    return logger


logger = configurar_logging()

In [ ]:
# =============================================================================
# CONFIGURACIÓN DE RUTAS
# =============================================================================

@dataclass
class ConfiguracionRutas:
    """
    Configuración centralizada de rutas del proyecto.

    Descubre automáticamente la estructura de directorios en TFM/ en lugar
    de asumir nombres fijos.

    Attributes:
        ruta_base: Directorio raíz del proyecto TFM
        ruta_imagenes: Directorio con imágenes originales
        ruta_caracteristicas: Directorio con JSONs de características
        ruta_gt: Directorio con máscaras ground truth de CVAT
        ruta_modelos: Directorio base de resultados de modelos
        ruta_analisis: Directorio de salida para análisis (Notebook 3)
    """
    ruta_base: Path
    ruta_imagenes: Path
    ruta_caracteristicas: Path
    ruta_gt: Path
    ruta_modelos: Path
    ruta_analisis: Path

    @classmethod
    def descubrir_desde_base(cls, ruta_base: str) -> 'ConfiguracionRutas':
        """
        Descubre automáticamente la estructura de directorios en TFM.

        Busca directorios que cumplan ciertos patrones en lugar de asumir
        nombres exactos. Esto hace el código más robusto ante variaciones
        en la estructura del proyecto.

        Args:
            ruta_base: Ruta al directorio TFM (ej: '/content/drive/MyDrive/TFM')

        Returns:
            ConfiguracionRutas con rutas descubiertas

        Raises:
            FileNotFoundError: Si no se encuentra algún directorio crítico
        """
        base = Path(ruta_base)

        if not base.exists():
            raise FileNotFoundError(
                f"Directorio base no existe: {base}\n"
                f"Verifica que la ruta al proyecto TFM sea correcta."
            )

        logger.info(f"Escaneando estructura en: {base}")

        # Descubrir directorio de imágenes
        ruta_imagenes = cls._buscar_directorio(
            base,
            ['0_Imagenes', 'Imagenes', '0_imagenes', 'imagenes'],
            'Imágenes originales'
        )

        # Descubrir directorio de características
        ruta_caracteristicas = cls._buscar_directorio(
            base,
            ['1_Caracteristicas', 'Caracteristicas', '1_caracteristicas', 'caracteristicas'],
            'Características fotográficas'
        )

        # Descubrir directorio de ground truth
        ruta_gt = None
        posibles_gt = [
            base / '0_Imagenes_CVAT' / 'ground_truth_masks',
            base / 'Imagenes_CVAT' / 'ground_truth_masks',
            base / '0_Imagenes_CVAT' / 'ground_truth',
            base / 'ground_truth_masks',
            base / 'ground_truth',
            base / 'GT'
        ]

        for ruta in posibles_gt:
            if ruta.exists() and ruta.is_dir():
                ruta_gt = ruta
                logger.info(f"  Ground Truth encontrado en: {ruta.relative_to(base)}")
                break

        if ruta_gt is None:
            raise FileNotFoundError(
                f"No se encontró directorio de Ground Truth.\n"
                f"Buscado en: {[str(p.relative_to(base)) for p in posibles_gt]}\n"
                f"Verifica que las máscaras GT estén en alguna de estas ubicaciones."
            )

        # Descubrir directorio de modelos
        ruta_modelos = cls._buscar_directorio(
            base,
            ['2_Modelos', 'Modelos', '2_modelos', 'modelos'],
            'Resultados de modelos'
        )

        # Crear directorio de análisis si no existe
        ruta_analisis = base / '3_Analisis'
        ruta_analisis.mkdir(parents=True, exist_ok=True)
        logger.info(f"  Directorio de análisis: {ruta_analisis.relative_to(base)}")

        return cls(
            ruta_base=base,
            ruta_imagenes=ruta_imagenes,
            ruta_caracteristicas=ruta_caracteristicas,
            ruta_gt=ruta_gt,
            ruta_modelos=ruta_modelos,
            ruta_analisis=ruta_analisis
        )

    @staticmethod
    def _buscar_directorio(base: Path, nombres_posibles: List[str], descripcion: str) -> Path:
        """
        Busca un directorio con varios nombres posibles.

        Args:
            base: Directorio base donde buscar
            nombres_posibles: Lista de nombres a probar
            descripcion: Descripción para mensajes de error

        Returns:
            Path al directorio encontrado

        Raises:
            FileNotFoundError: Si no se encuentra ninguno
        """
        for nombre in nombres_posibles:
            ruta = base / nombre
            if ruta.exists() and ruta.is_dir():
                logger.info(f"  {descripcion} encontrado en: {ruta.relative_to(base)}")
                return ruta

        # Si llegamos aquí, no se encontró
        raise FileNotFoundError(
            f"No se encontró directorio de {descripcion}.\n"
            f"Buscado en: {nombres_posibles}\n"
            f"Verifica la estructura del proyecto en: {base}"
        )

    def validar(self) -> Tuple[bool, List[str]]:
        """
        Valida existencia de directorios críticos.

        Returns:
            Tupla (todas_existen, lista_errores)
        """
        errores = []

        rutas_criticas = [
            ('Base', self.ruta_base),
            ('Imágenes', self.ruta_imagenes),
            ('Características', self.ruta_caracteristicas),
            ('Ground Truth', self.ruta_gt),
            ('Modelos', self.ruta_modelos)
        ]

        for nombre, ruta in rutas_criticas:
            if not ruta.exists():
                errores.append(f"Ruta {nombre} no existe: {ruta}")

        return len(errores) == 0, errores

    def mostrar_estructura(self):
        """Muestra la estructura de rutas descubiertas."""
        print("\nEstructura de rutas descubiertas:")
        print(f"  Base:             {self.ruta_base}")
        print(f"  Imágenes:         {self.ruta_imagenes}")
        print(f"  Características:  {self.ruta_caracteristicas}")
        print(f"  Ground Truth:     {self.ruta_gt}")
        print(f"  Modelos:          {self.ruta_modelos}")
        print(f"  Análisis:         {self.ruta_analisis}")

In [ ]:
# =============================================================================
# ESTRUCTURA DE DATOS CONSOLIDADA
# =============================================================================

@dataclass
class DatosFotografia:
    """
    Estructura consolidada de todos los datos de una fotografía.

    Integra características fotográficas, ground truth, máscaras de todos
    los modelos evaluados y metadata de ejecución.

    Attributes:
        foto_id: Identificador único (nombre sin extensión, ej: '_DSC0002')
        ruta_imagen: Path a imagen original
        ruta_json_caracteristicas: Path a JSON de características
        ruta_gt: Path a máscara ground truth NPZ

        metadatos: Información de archivo (dimensiones, tamaño)
        exif_disponible: Flag indicando si EXIF está presente
        exif: Diccionario con metadatos EXIF (vacío si no disponible)
        color: Estadísticas de color (RGB, HSV, LAB)
        saliencia: Análisis de saliencia visual
        calidad: Métricas de calidad de imagen

        gt_disponible: Flag indicando si ground truth existe
        gt_mascara: Máscara ground truth binaria (H, W)
        gt_shape: Dimensiones de la máscara GT

        mascaras_modelos: Dict {codigo_config: datos_mascara}
        metadata_modelos: Dict {codigo_config: metadata_ejecucion}

        timestamp_integracion: Timestamp de procesamiento
        num_configs_disponibles: Número de configuraciones de modelos cargadas
    """
    foto_id: str

    # Rutas de archivos
    ruta_imagen: Path
    ruta_json_caracteristicas: Path
    ruta_gt: Optional[Path]

    # Características intrínsecas
    metadatos: Dict[str, Any]
    exif_disponible: bool
    exif: Dict[str, Any]
    color: Dict[str, Any]
    saliencia: Dict[str, Any]
    calidad: Dict[str, Any]

    # Ground Truth
    gt_disponible: bool
    gt_mascara: Optional[np.ndarray]
    gt_shape: Optional[Tuple[int, int]]

    # Máscaras y metadata de modelos
    mascaras_modelos: Dict[str, Dict[str, Any]]
    metadata_modelos: Dict[str, Dict[str, Any]]

    # Metadata de procesamiento
    timestamp_integracion: str
    num_configs_disponibles: int

    def a_dict(self) -> Dict[str, Any]:
        """
        Convierte a diccionario serializable (sin arrays numpy).

        Returns:
            Dict con estructura completa excepto máscaras numpy
        """
        return {
            'foto_id': self.foto_id,
            'rutas': {
                'imagen': str(self.ruta_imagen),
                'caracteristicas': str(self.ruta_json_caracteristicas),
                'ground_truth': str(self.ruta_gt) if self.ruta_gt else None
            },
            'metadatos': self.metadatos,
            'exif_disponible': self.exif_disponible,
            'exif': self.exif,
            'color': self.color,
            'saliencia': self.saliencia,
            'calidad': self.calidad,
            'ground_truth': {
                'disponible': self.gt_disponible,
                'shape': self.gt_shape
            },
            'modelos': {
                'num_configuraciones': self.num_configs_disponibles,
                'configuraciones': list(self.mascaras_modelos.keys()),
                'metadata_disponible': list(self.metadata_modelos.keys())
            },
            'timestamp': self.timestamp_integracion
        }


In [ ]:
# =============================================================================
# LECTOR DE CARACTERÍSTICAS FOTOGRÁFICAS
# =============================================================================

class LectorCaracteristicas:
    """
    Carga y procesa características fotográficas del Notebook 00.

    Implementa manejo defensivo de EXIF que puede estar vacío en las
    fotografías reales del dataset. Busca JSONs en subdirectorio json/
    si existe.
    """

    def __init__(self, ruta_caracteristicas: Path):
        """
        Args:
            ruta_caracteristicas: Path al directorio de JSONs de características
        """
        self.ruta_caracteristicas = ruta_caracteristicas

        # Verificar si los JSONs están en subdirectorio 'json/'
        self.ruta_json_subdir = ruta_caracteristicas / 'json'
        self.usar_subdirectorio = self.ruta_json_subdir.exists()

        if self.usar_subdirectorio:
            logger.info(f"  JSONs encontrados en subdirectorio: {self.ruta_json_subdir.relative_to(ruta_caracteristicas.parent)}")

    def cargar(self, foto_id: str) -> Optional[Dict[str, Any]]:
        """
        Carga características de una fotografía desde JSON.

        Busca primero en subdirectorio json/, luego en el directorio raíz.

        Args:
            foto_id: Identificador de la foto (ej: '_DSC0002')

        Returns:
            Dict con características extraídas o None si error crítico

        Raises:
            FileNotFoundError: Si el JSON no existe (error crítico)
        """
        # Intentar primero en subdirectorio json/
        if self.usar_subdirectorio:
            ruta_json = self.ruta_json_subdir / f"{foto_id}_caracteristicas.json"
        else:
            ruta_json = self.ruta_caracteristicas / f"{foto_id}_caracteristicas.json"

        # Si no existe, intentar en el directorio raíz
        if not ruta_json.exists():
            ruta_alternativa = self.ruta_caracteristicas / f"{foto_id}_caracteristicas.json"
            if ruta_alternativa.exists():
                ruta_json = ruta_alternativa
            else:
                # Último intento: buscar en subdirectorio json/
                ruta_json_alt = self.ruta_caracteristicas / 'json' / f"{foto_id}_caracteristicas.json"
                if ruta_json_alt.exists():
                    ruta_json = ruta_json_alt
                else:
                    raise FileNotFoundError(
                        f"JSON de características no encontrado: {ruta_json}\n"
                        f"También se buscó en: {ruta_alternativa}\n"
                        f"Y en: {ruta_json_alt}\n"
                        f"Este es un error crítico. La foto {foto_id} debe tener "
                        f"características generadas por Notebook 00."
                    )

        try:
            with open(ruta_json, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Extraer campos con valores por defecto
            caracteristicas = {
                'ruta_json': ruta_json,
                'metadatos': self._extraer_metadatos(data),
                'exif_disponible': self._verificar_exif(data),
                'exif': self._extraer_exif(data),
                'color': self._extraer_color(data),
                'saliencia': self._extraer_saliencia(data),
                'calidad': self._extraer_calidad(data)
            }

            return caracteristicas

        except json.JSONDecodeError as e:
            raise ValueError(
                f"JSON corrupto para {foto_id}: {e}\n"
                f"Este es un error crítico. El JSON debe ser válido."
            )
        except Exception as e:
            logger.error(f"Error inesperado cargando {foto_id}: {e}")
            raise

    def _extraer_metadatos(self, data: Dict) -> Dict[str, Any]:
        """Extrae metadatos de archivo con manejo de campos faltantes."""
        metadatos_raw = data.get('metadatos_archivo', {})

        return {
            'nombre': metadatos_raw.get('nombre_archivo', 'desconocido'),
            'ancho': metadatos_raw.get('ancho_original', 0),
            'alto': metadatos_raw.get('alto_original', 0),
            'tamaño_mb': metadatos_raw.get('tamaño_mb', 0.0),
            'fecha_modificacion': metadatos_raw.get('fecha_modificacion', None),
            'modo_color': metadatos_raw.get('modo_color', 'RGB')
        }

    def _verificar_exif(self, data: Dict) -> bool:
        """
        Verifica si hay datos EXIF disponibles.

        En el dataset real, metadatos_exif es un diccionario vacío.
        """
        exif = data.get('metadatos_exif', {})
        return len(exif) > 0

    def _extraer_exif(self, data: Dict) -> Dict[str, Any]:
        """
        Extrae metadatos EXIF con manejo defensivo.

        Nota: En el dataset real, este diccionario está vacío.
        Se mantiene la estructura para compatibilidad futura.
        """
        exif = data.get('metadatos_exif', {})

        # Si está vacío, retornar estructura con valores None
        if len(exif) == 0:
            return {
                'aperture': None,
                'iso': None,
                'shutter_speed': None,
                'focal_length': None,
                'camera': None,
                'lens': None
            }

        # Si hay datos, extraer con valores por defecto
        return {
            'aperture': exif.get('aperture', None),
            'iso': exif.get('iso', None),
            'shutter_speed': exif.get('shutter_speed', None),
            'focal_length': exif.get('focal_length', None),
            'camera': exif.get('camera', None),
            'lens': exif.get('lens', None)
        }

    def _extraer_color(self, data: Dict) -> Dict[str, Any]:
        """Extrae estadísticas de color."""
        color_data = data.get('estadisticas_color', {})

        rgb = color_data.get('rgb', {})
        hsv = color_data.get('hsv', {})
        lab = color_data.get('lab', {})
        global_stats = color_data.get('global', {})

        return {
            'rgb_mean': rgb.get('mean', [0, 0, 0]),
            'rgb_stddev': rgb.get('stddev', [0, 0, 0]),
            'hsv_saturation_mean': hsv.get('saturation_mean', 0.0),
            'hsv_value_mean': hsv.get('value_mean', 0.0),
            'lab_l_mean': lab.get('l_mean', 0.0),
            'brillo_promedio': global_stats.get('brillo_promedio', 0.0),
            'contraste': global_stats.get('contraste', 0.0),
            'entropia': global_stats.get('entropia', 0.0)
        }

    def _extraer_saliencia(self, data: Dict) -> Dict[str, Any]:
        """Extrae análisis de saliencia visual."""
        saliencia_data = data.get('saliencia_visual', {})
        centroide = saliencia_data.get('centroide', {})

        return {
            'centroide_x': centroide.get('centroide_x_normalizado', 0.5),
            'centroide_y': centroide.get('centroide_y_normalizado', 0.5),
            'dist_centro': centroide.get('distancia_desde_centro', 0.0),
            'area_saliente_pct': centroide.get('area_saliente_porcentaje', 0.0)
        }

    def _extraer_calidad(self, data: Dict) -> Dict[str, Any]:
        """Extrae métricas de calidad de imagen."""
        calidad_data = data.get('calidad', {})

        nitidez = calidad_data.get('nitidez', {})
        ruido = calidad_data.get('ruido', {})
        exposicion = calidad_data.get('exposicion', {})

        return {
            'nitidez_laplacian': nitidez.get('laplacian', 0.0),
            'nitidez_tenengrad': nitidez.get('tenengrad', 0.0),
            'snr_db': ruido.get('snr_db', 0.0),
            'brillo': exposicion.get('brillo_medio', 0.0),
            'sobre_expuesto_pct': exposicion.get('sobre_expuesto_pct', 0.0),
            'sub_expuesto_pct': exposicion.get('sub_expuesto_pct', 0.0)
        }

In [ ]:
# =============================================================================
# LECTOR DE GROUND TRUTH
# =============================================================================

class LectorGroundTruth:
    """
    Carga máscaras ground truth anotadas manualmente en CVAT.

    Las máscaras GT son críticas para el análisis comparativo. Su ausencia
    genera error crítico que detiene la ejecución.
    """

    def __init__(self, ruta_gt: Path):
        """
        Args:
            ruta_gt: Path al directorio de máscaras ground truth NPZ
        """
        self.ruta_gt = ruta_gt

    def cargar(self, foto_id: str) -> Dict[str, Any]:
        """
        Carga máscara ground truth para una fotografía.

        Args:
            foto_id: Identificador de la foto

        Returns:
            Dict con máscara y metadata

        Raises:
            FileNotFoundError: Si la máscara GT no existe (error crítico)
        """
        ruta_npz = self.ruta_gt / f"{foto_id}_gt.npz"

        if not ruta_npz.exists():
            raise FileNotFoundError(
                f"Ground Truth no encontrado: {ruta_npz}\n"
                f"Este es un error crítico. Todas las fotografías deben tener "
                f"anotación manual en CVAT para evaluación cuantitativa."
            )

        try:
            data = np.load(ruta_npz)

            # Extraer máscara (campo 'masks' según generación CVAT)
            if 'masks' not in data:
                raise ValueError(
                    f"NPZ de GT no contiene campo 'masks': {ruta_npz}\n"
                    f"Campos disponibles: {list(data.keys())}"
                )

            mascara = data['masks']

            # Convertir a binaria si es necesario (puede estar en [0, 1] float)
            if mascara.dtype == np.float32 or mascara.dtype == np.float64:
                mascara_binaria = (mascara > 0.5).astype(np.uint8)
            else:
                mascara_binaria = mascara.astype(np.uint8)

            # Validar que sea binaria
            valores_unicos = np.unique(mascara_binaria)
            if not np.array_equal(valores_unicos, [0, 1]) and not np.array_equal(valores_unicos, [0]) and not np.array_equal(valores_unicos, [1]):
                logger.warning(
                    f"Máscara GT de {foto_id} no es estrictamente binaria. "
                    f"Valores únicos: {valores_unicos}"
                )

            return {
                'existe': True,
                'mascara': mascara_binaria,
                'shape': mascara_binaria.shape,
                'ruta': ruta_npz,
                'area_pixels': int(np.sum(mascara_binaria)),
                'cobertura_pct': float(np.mean(mascara_binaria) * 100)
            }

        except Exception as e:
            raise RuntimeError(
                f"Error crítico cargando GT de {foto_id}: {e}"
            )


In [ ]:
# =============================================================================
# ESCANEO DE CONFIGURACIONES DE MODELOS
# =============================================================================

class EscanerConfiguraciones:
    """
    Escanea dinámicamente configuraciones de modelos disponibles.

    Busca recursivamente carpetas 'mascaras' en la estructura de cada modelo,
    ya que diferentes modelos tienen estructuras de directorios distintas.
    """

    MODELOS = ['mask2former', 'oneformer', 'sam2', 'yolov8', 'bodypix']

    # Mapeo de códigos cortos para nomenclatura GitHub-safe
    CODIGOS_MODELO = {
        'mask2former': 'm2f',
        'oneformer': 'o1f',
        'sam2': 'sam',
        'yolov8': 'yolo',
        'bodypix': 'bpx'
    }

    def __init__(self, ruta_modelos: Path):
        """
        Args:
            ruta_modelos: Path al directorio base de modelos (/TFM/2_Modelos)
        """
        self.ruta_modelos = ruta_modelos

    def escanear(self) -> Dict[str, List[Tuple[str, Path]]]:
        """
        Escanea directorios de modelos y retorna configuraciones disponibles.

        Returns:
            Dict {modelo: [(nombre_config, ruta_mascaras), ...]}
        """
        configs_por_modelo = {}

        for modelo in self.MODELOS:
            ruta_modelo = self.ruta_modelos / modelo

            if not ruta_modelo.exists():
                logger.warning(
                    f"Directorio de modelo no existe: {modelo}. "
                    f"No se cargarán máscaras de este modelo."
                )
                configs_por_modelo[modelo] = []
                continue

            # Buscar recursivamente carpetas 'mascaras'
            configs = self._buscar_configuraciones_recursivo(ruta_modelo, modelo)

            configs_por_modelo[modelo] = configs

            logger.info(
                f"Modelo {modelo}: {len(configs)} configuraciones encontradas"
            )

            # Mostrar cada configuración encontrada
            if len(configs) > 0 and len(configs) <= 20:  # Solo si no son demasiadas
                logger.info(f"  Configuraciones de {modelo}:")
                for nombre_config, ruta_mascaras in configs:
                    # Mostrar ruta relativa desde el modelo
                    ruta_relativa = ruta_mascaras.relative_to(ruta_modelo)
                    logger.info(f"    - {nombre_config}: {ruta_relativa}")

        return configs_por_modelo

    def _buscar_configuraciones_recursivo(self, ruta_base: Path, modelo: str, max_depth: int = 4) -> List[Tuple[str, Path]]:
        """
        Busca recursivamente carpetas 'mascaras' hasta una profundidad máxima.

        Args:
            ruta_base: Directorio base del modelo
            modelo: Nombre del modelo
            max_depth: Profundidad máxima de búsqueda

        Returns:
            Lista de tuplas (nombre_config, ruta_mascaras)
        """
        configs = []

        def buscar_recursivo(ruta_actual: Path, profundidad: int, ruta_relativa: str = ""):
            if profundidad > max_depth:
                return

            try:
                for item in ruta_actual.iterdir():
                    if not item.is_dir():
                        continue

                    # Si encontramos carpeta 'mascaras'
                    if item.name == 'mascaras':
                        # Generar nombre de configuración desde la ruta relativa
                        if ruta_relativa:
                            nombre_config = ruta_relativa.replace('/', '_')
                        else:
                            # Si está directamente en el modelo
                            nombre_config = 'default'

                        configs.append((nombre_config, item))
                        continue

                    # Continuar buscando en subdirectorios
                    nueva_ruta_relativa = f"{ruta_relativa}/{item.name}" if ruta_relativa else item.name
                    buscar_recursivo(item, profundidad + 1, nueva_ruta_relativa)
            except PermissionError:
                # Ignorar directorios sin permisos
                pass

        buscar_recursivo(ruta_base, 0)
        return configs

    def generar_codigo_config(self, modelo: str, config: str) -> str:
        """
        Genera código corto GitHub-safe para una configuración.

        Args:
            modelo: Nombre del modelo
            config: Nombre de la configuración

        Returns:
            Código único, ej: "m2f_large_ade_baja"
        """
        codigo_modelo = self.CODIGOS_MODELO.get(modelo, modelo[:3])

        # Limpiar nombre de configuración
        config_limpio = config.replace('_resultados', '').replace('resultados_', '')

        # Combinar y truncar si es necesario
        codigo_completo = f"{codigo_modelo}_{config_limpio}"

        if len(codigo_completo) > 50:
            max_config = 50 - len(codigo_modelo) - 1
            config_truncado = config_limpio[:max_config]
            codigo_completo = f"{codigo_modelo}_{config_truncado}"

        return codigo_completo

In [ ]:
# =============================================================================
# LECTOR DE MÁSCARAS DE MODELOS
# =============================================================================

class LectorMascarasModelos:
    """
    Carga máscaras NPZ de modelos con manejo robusto de estructuras heterogéneas.

    Cada modelo guarda diferentes campos en sus archivos NPZ. Esta clase
    implementa detección automática de campos disponibles y extracción
    defensiva con try/except.

    Criterio de error: Falta de máscara de un modelo específico es WARNING,
    no ERROR CRÍTICO (puede que un modelo no haya procesado esa foto).
    """

    def __init__(self, ruta_modelos: Path, escaner: EscanerConfiguraciones):
        """
        Args:
            ruta_modelos: Path al directorio base de modelos
            escaner: Instancia de EscanerConfiguraciones para códigos
        """
        self.ruta_modelos = ruta_modelos
        self.escaner = escaner
        # Guardar mapeo de configs a rutas de máscaras
        self.configs_rutas = {}

    def registrar_configuracion(self, modelo: str, nombre_config: str, ruta_mascaras: Path):
        """
        Registra la ruta de máscaras para una configuración.

        Args:
            modelo: Nombre del modelo
            nombre_config: Nombre de la configuración
            ruta_mascaras: Path al directorio de máscaras
        """
        codigo = self.escaner.generar_codigo_config(modelo, nombre_config)
        self.configs_rutas[codigo] = {
            'modelo': modelo,
            'config': nombre_config,
            'ruta_mascaras': ruta_mascaras
        }

    def cargar(self, codigo_config: str, foto_id: str) -> Optional[Dict[str, Any]]:
        """
        Carga máscara NPZ usando el código de configuración.

        Args:
            codigo_config: Código de configuración generado
            foto_id: Identificador de la foto

        Returns:
            Dict con máscara y metadata disponible, o None si no existe
        """
        if codigo_config not in self.configs_rutas:
            logger.warning(f"Configuración no registrada: {codigo_config}")
            return None

        config_info = self.configs_rutas[codigo_config]
        ruta_mascaras = config_info['ruta_mascaras']
        modelo = config_info['modelo']

        # Buscar el archivo NPZ para esta foto
        ruta_npz = self._buscar_npz(ruta_mascaras, foto_id)

        if ruta_npz is None:
            return None

        try:
            data = np.load(ruta_npz, allow_pickle=True)

            # Intentar extraer máscara con diferentes nombres posibles
            mascara = self._extraer_mascara(data, modelo)

            if mascara is None:
                logger.warning(
                    f"No se pudo extraer máscara de {codigo_config}/{foto_id}. "
                    f"Campos disponibles: {list(data.keys())}"
                )
                return None

            # Convertir a binaria uint8 si no lo es
            if mascara.dtype != np.uint8:
                mascara = (mascara > 0.5).astype(np.uint8)

            # Construir resultado con campos opcionales
            resultado = {
                'mascara': mascara,
                'shape': mascara.shape,
                'ruta': ruta_npz,
                'campos_disponibles': list(data.keys())
            }

            # Extraer campos opcionales (no críticos)
            resultado['bbox'] = self._extraer_bbox(data)
            resultado['score'] = self._extraer_score(data)
            resultado['area'] = self._extraer_area(data, mascara)

            return resultado

        except Exception as e:
            logger.error(
                f"Error cargando máscara {codigo_config}/{foto_id}: {e}"
            )
            return None

    def _buscar_npz(self, ruta_mascaras: Path, foto_id: str) -> Optional[Path]:
        """
        Busca el archivo NPZ de una foto en múltiples ubicaciones posibles.

        Args:
            ruta_mascaras: Path al directorio de máscaras
            foto_id: Identificador de la foto

        Returns:
            Path al NPZ encontrado o None
        """
        # Intento 1: Directamente en mascaras/{foto_id}.npz
        ruta_directa = ruta_mascaras / f"{foto_id}.npz"
        if ruta_directa.exists():
            return ruta_directa

        # Intento 2: En subdirectorio con nombre de foto
        subdir_foto = ruta_mascaras / foto_id
        if subdir_foto.exists() and subdir_foto.is_dir():
            # Buscar NPZ directamente en el subdirectorio
            npz_directo = subdir_foto / f"{foto_id}.npz"
            if npz_directo.exists():
                return npz_directo

            npz_simple = subdir_foto / "mascara.npz"
            if npz_simple.exists():
                return npz_simple

            # Buscar en subdirectorios de umbrales (Mask2Former)
            for item in subdir_foto.iterdir():
                if item.is_dir():
                    # Buscar primer NPZ en este directorio
                    npz_files = list(item.glob("*.npz"))
                    if npz_files:
                        # Tomar el primer archivo (o el que tenga mayor número)
                        return sorted(npz_files)[0]

        # Intento 3: Buscar recursivamente (último recurso)
        npz_files = list(ruta_mascaras.rglob(f"*{foto_id}*.npz"))
        if npz_files:
            return npz_files[0]

        return None

    def _extraer_mascara(self, data: np.lib.npyio.NpzFile, modelo: str) -> Optional[np.ndarray]:
        """
        Extrae máscara binaria con lógica específica por modelo.

        Estrategia:
        - mask2former: 'mascara' o 'mask'
        - sam2: 'mascaras_binarias' (stack, tomar primera)
        - yolov8: 'masks' (stack, tomar primera)
        - oneformer: similar a mask2former
        - bodypix: 'mascara_probabilidad' con threshold o mascaras_binarias_por_umbral
        """
        # Intentar nombres comunes primero
        for nombre in ['mascara', 'mask', 'segmentation']:
            if nombre in data:
                mascara = data[nombre]
                # Si es un array válido, retornarlo
                if isinstance(mascara, np.ndarray) and mascara.ndim >= 2:
                    return mascara

        # Modelos con stacks (SAM2, YOLO)
        if 'mascaras_binarias' in data:  # SAM2
            stack = data['mascaras_binarias']
            if isinstance(stack, np.ndarray):
                if stack.ndim == 3 and stack.shape[0] > 0:
                    return stack[0]
                elif stack.ndim == 2:
                    return stack

        if 'masks' in data:  # YOLO o general
            stack = data['masks']
            if isinstance(stack, np.ndarray):
                if stack.ndim == 3 and stack.shape[0] > 0:
                    return stack[0]
                elif stack.ndim == 2:
                    return stack

        # BodyPix con umbrales
        if 'mascaras_binarias_por_umbral' in data:
            # Tomar umbral medio (suele ser el más balanceado)
            try:
                umbrales_dict = data['mascaras_binarias_por_umbral'].item()
                if isinstance(umbrales_dict, dict) and len(umbrales_dict) > 0:
                    umbral_medio = sorted(umbrales_dict.keys())[len(umbrales_dict)//2]
                    return umbrales_dict[umbral_medio]
            except:
                pass

        if 'mascara_probabilidad' in data:
            # Threshold en 0.5
            prob_mask = data['mascara_probabilidad']
            if isinstance(prob_mask, np.ndarray):
                return (prob_mask > 0.5).astype(np.uint8)

        return None

    def _extraer_bbox(self, data: np.lib.npyio.NpzFile) -> Optional[np.ndarray]:
        """Extrae bounding box si está disponible."""
        for nombre in ['bbox', 'bboxes', 'boxes']:
            if nombre in data:
                try:
                    bbox_data = data[nombre]
                    if isinstance(bbox_data, np.ndarray):
                        if bbox_data.ndim == 2 and bbox_data.shape[0] > 0:
                            return bbox_data[0]
                        elif bbox_data.ndim == 1:
                            return bbox_data
                except:
                    continue
        return None

    def _extraer_score(self, data: np.lib.npyio.NpzFile) -> Optional[float]:
        """Extrae score de confianza si está disponible."""
        for nombre in ['score', 'scores', 'confidence', 'person_confidence']:
            if nombre in data:
                try:
                    score_data = data[nombre]
                    if isinstance(score_data, np.ndarray):
                        if score_data.size > 0:
                            return float(score_data.flat[0])
                    else:
                        return float(score_data)
                except:
                    continue
        return None

    def _extraer_area(self, data: np.lib.npyio.NpzFile, mascara: np.ndarray) -> int:
        """Extrae área o la calcula desde la máscara."""
        for nombre in ['area', 'areas']:
            if nombre in data:
                try:
                    area_data = data[nombre]
                    if isinstance(area_data, np.ndarray):
                        if area_data.size > 0:
                            return int(area_data.flat[0])
                    else:
                        return int(area_data)
                except:
                    continue

        # Calcular desde máscara si no está disponible
        return int(np.sum(mascara))

In [ ]:
# =============================================================================
# LECTOR DE METADATA DE MODELOS
# =============================================================================

class LectorMetadataModelos:
    """
    Carga JSONs de metadata de ejecución de modelos.

    Estos JSONs contienen información valiosa sobre:
    - Tiempos de inferencia y procesamiento
    - Scores y confianzas por umbral
    - Configuraciones específicas del modelo
    - Dimensiones procesadas y factor de escala
    - Estrategias de prompts (SAM2)
    - Partes del cuerpo (BodyPix)
    """

    def __init__(self, ruta_modelos: Path):
        """
        Args:
            ruta_modelos: Path al directorio base de modelos
        """
        self.ruta_modelos = ruta_modelos
        # Mapeo de códigos de configuración a rutas de JSONs
        self.rutas_json = {}

    def registrar_ruta_json(self, codigo_config: str, ruta_mascaras: Path):
        """
        Registra dónde buscar JSONs para una configuración.

        Args:
            codigo_config: Código de la configuración
            ruta_mascaras: Path al directorio de máscaras
        """
        # Buscar directorio json/ al mismo nivel o en el padre
        ruta_json = None

        # Intento 1: Al mismo nivel (resultados/json y resultados/mascaras)
        dir_padre = ruta_mascaras.parent
        json_dir = dir_padre / 'json'
        if json_dir.exists():
            ruta_json = json_dir
        else:
            # Intento 2: En subdirectorio del padre
            for item in dir_padre.rglob('json'):
                if item.is_dir():
                    ruta_json = item
                    break

        if ruta_json:
            self.rutas_json[codigo_config] = ruta_json

    def cargar(self, codigo_config: str, foto_id: str) -> Optional[Dict[str, Any]]:
        """
        Carga metadata JSON de ejecución de un modelo para una foto.

        Args:
            codigo_config: Código de configuración
            foto_id: Identificador de la foto

        Returns:
            Dict con metadata o None si no existe
        """
        if codigo_config not in self.rutas_json:
            return None

        ruta_json_dir = self.rutas_json[codigo_config]

        # Buscar JSON con el foto_id (puede tener timestamp)
        json_files = list(ruta_json_dir.glob(f"{foto_id}*.json"))

        if len(json_files) == 0:
            return None

        # Si hay múltiples, tomar el más reciente
        if len(json_files) > 1:
            json_files.sort(key=lambda x: x.stat().st_mtime, reverse=True)

        ruta_json = json_files[0]

        try:
            with open(ruta_json, 'r', encoding='utf-8') as f:
                metadata = json.load(f)

            # Extraer campos relevantes de forma unificada
            metadata_unificada = self._unificar_metadata(metadata, codigo_config)
            metadata_unificada['ruta_json'] = str(ruta_json)

            return metadata_unificada

        except Exception as e:
            logger.warning(f"Error cargando metadata {codigo_config}/{foto_id}: {e}")
            return None

    def _unificar_metadata(self, metadata: Dict, codigo_config: str) -> Dict[str, Any]:
        """
        Unifica metadata de diferentes modelos en estructura común.

        Args:
            metadata: Metadata raw del JSON
            codigo_config: Código de configuración para detectar modelo

        Returns:
            Dict con campos unificados
        """
        unificado = {
            'metadata_raw': metadata,  # Guardar original completo
            'modelo': None,
            'timestamp': None,
            'tiempos': {},
            'dimensiones': {},
            'scores': {},
            'detecciones': {},
            'configuracion': {}
        }

        # Detectar tipo de modelo por código
        if 'm2f' in codigo_config or 'mask2former' in str(metadata.get('metadata', {}).get('modelo', '')).lower():
            unificado = self._extraer_mask2former(metadata, unificado)
        elif 'o1f' in codigo_config or 'oneformer' in str(metadata.get('metadata', {}).get('modelo', '')).lower():
            unificado = self._extraer_oneformer(metadata, unificado)
        elif 'sam' in codigo_config or 'sam2' in str(metadata.get('modelo', '')).lower():
            unificado = self._extraer_sam2(metadata, unificado)
        elif 'yolo' in codigo_config:
            unificado = self._extraer_yolo(metadata, unificado)
        elif 'bpx' in codigo_config or 'bodypix' in str(metadata.get('metadata', {}).get('modelo', {})).lower():
            unificado = self._extraer_bodypix(metadata, unificado)

        return unificado

    def _extraer_mask2former(self, metadata: Dict, unificado: Dict) -> Dict:
        """Extrae campos específicos de Mask2Former."""
        meta = metadata.get('metadata', {})

        unificado['modelo'] = meta.get('modelo', 'mask2former')
        unificado['timestamp'] = meta.get('timestamp')

        # Tiempos
        rendimiento = metadata.get('rendimiento', {})
        unificado['tiempos'] = {
            'inferencia_ms': rendimiento.get('tiempo_inferencia_ms'),
            'total_ms': rendimiento.get('tiempo_total_ms'),
            'gpu_pico_mb': rendimiento.get('gpu_pico_mb')
        }

        # Dimensiones
        dims = meta.get('dimensiones', {})
        unificado['dimensiones'] = {
            'original': dims.get('original', {}),
            'procesada': dims.get('procesada', {}),
            'redimensionada': dims.get('procesada', {}).get('redimensionada', False)
        }

        # Scores y detecciones por umbral
        detecciones = metadata.get('detecciones_por_umbral', {})
        unificado['detecciones'] = {
            'por_umbral': {},
            'num_umbrales': len(detecciones)
        }

        for umbral_key, umbral_data in detecciones.items():
            unificado['detecciones']['por_umbral'][umbral_key] = {
                'num_personas': umbral_data.get('num_personas_detectadas', 0),
                'scores': umbral_data.get('scores', []),
                'score_promedio': umbral_data.get('score_promedio', 0.0),
                'score_maximo': umbral_data.get('score_maximo', 0.0)
            }

        # Configuración
        config = meta.get('config_umbrales', {})
        unificado['configuracion'] = {
            'nombre': config.get('nombre'),
            'umbrales': config.get('valores', []),
            'descripcion': config.get('descripcion')
        }

        return unificado

    def _extraer_oneformer(self, metadata: Dict, unificado: Dict) -> Dict:
        """Extrae campos específicos de OneFormer (similar a Mask2Former)."""
        return self._extraer_mask2former(metadata, unificado)

    def _extraer_sam2(self, metadata: Dict, unificado: Dict) -> Dict:
        """Extrae campos específicos de SAM2."""
        unificado['modelo'] = metadata.get('modelo', 'sam2')
        unificado['timestamp'] = metadata.get('timestamp')

        # Tiempos
        proc = metadata.get('procesamiento', {})
        unificado['tiempos'] = {
            'carga_ms': proc.get('tiempo_carga_ms'),
            'set_imagen_ms': proc.get('tiempo_set_imagen_ms'),
            'prediccion_ms': proc.get('tiempo_prediccion_ms'),
            'total_ms': proc.get('tiempo_total_ms')
        }

        # Dimensiones
        dims = metadata.get('dimensiones', {})
        unificado['dimensiones'] = {
            'original': dims.get('original', {}),
            'procesada': dims.get('procesada', {}),
            'redimensionada': dims.get('procesada', {}).get('redimensionada', False)
        }

        # Prompts y estrategia
        prompts = metadata.get('prompts', {})
        unificado['configuracion'] = {
            'estrategia': prompts.get('estrategia'),
            'tipo_prompts': prompts.get('tipo'),
            'num_puntos': prompts.get('num_point_coords', 0),
            'num_boxes': prompts.get('num_boxes', 0)
        }

        # Scores y detecciones
        resultados = metadata.get('resultados', {})
        metricas = metadata.get('metricas', {})
        unificado['detecciones'] = {
            'mascaras_generadas': resultados.get('mascaras_generadas', 0),
            'personas_detectadas': resultados.get('personas_detectadas', 0)
        }
        unificado['scores'] = {
            'scores_sam': metricas.get('scores_sam', []),
            'scores_personas': metricas.get('scores_personas', [])
        }

        return unificado

    def _extraer_yolo(self, metadata: Dict, unificado: Dict) -> Dict:
        """Extrae campos específicos de YOLO."""
        # Implementar según estructura de YOLO cuando esté disponible
        unificado['modelo'] = 'yolov8-seg'
        return unificado

    def _extraer_bodypix(self, metadata: Dict, unificado: Dict) -> Dict:
        """Extrae campos específicos de BodyPix."""
        meta = metadata.get('metadata', {})
        modelo_info = meta.get('modelo', {})

        unificado['modelo'] = modelo_info.get('nombre', 'bodypix')
        unificado['timestamp'] = meta.get('timestamp')

        # Tiempos
        unificado['tiempos'] = {
            'inferencia_ms': meta.get('tiempo_inferencia_ms'),
            'total_ms': meta.get('tiempo_inferencia_ms')
        }

        # Dimensiones
        imagen_info = meta.get('imagen', {})
        unificado['dimensiones'] = {
            'original': {
                'width': imagen_info.get('ancho_original'),
                'height': imagen_info.get('alto_original')
            },
            'procesada': {
                'width': imagen_info.get('ancho_procesado'),
                'height': imagen_info.get('alto_procesado')
            },
            'redimensionada': imagen_info.get('redimensionado', False),
            'factor_escala': imagen_info.get('factor_escala')
        }

        # Detecciones por umbral
        detecciones = metadata.get('detecciones_por_umbral', {})
        unificado['detecciones'] = {
            'por_umbral': {},
            'num_umbrales': len(detecciones)
        }

        for umbral_key, umbral_data in detecciones.items():
            unificado['detecciones']['por_umbral'][umbral_key] = {
                'persona_detectada': umbral_data.get('persona_detectada', False),
                'area_pixels': umbral_data.get('area_pixels', 0),
                'porcentaje_imagen': umbral_data.get('porcentaje_imagen', 0.0),
                'probabilidad_media': umbral_data.get('probabilidad_media_region', 0.0)
            }

        # Partes del cuerpo (único de BodyPix)
        partes = metadata.get('partes_cuerpo', {})
        unificado['configuracion'] = {
            'partes_detectadas': partes.get('num_partes_detectadas', 0),
            'grupos_detectados': partes.get('num_grupos_detectados', 0),
            'grupos_disponibles': partes.get('grupos_disponibles', [])
        }

        return unificado

In [ ]:
# =============================================================================
# INTEGRADOR PRINCIPAL
# =============================================================================

class IntegradorDatos:
    """
    Orquesta la integración de todos los datos de una fotografía.

    Combina características fotográficas, ground truth, máscaras de todos
    los modelos y metadata de ejecución en estructura DatosFotografia unificada.
    """

    def __init__(self, config_rutas: ConfiguracionRutas):
        """
        Args:
            config_rutas: Configuración de rutas del proyecto
        """
        self.config = config_rutas

        # Inicializar lectores
        self.lector_caracteristicas = LectorCaracteristicas(
            config_rutas.ruta_caracteristicas
        )
        self.lector_gt = LectorGroundTruth(config_rutas.ruta_gt)
        self.escaner = EscanerConfiguraciones(config_rutas.ruta_modelos)
        self.lector_mascaras = LectorMascarasModelos(
            config_rutas.ruta_modelos,
            self.escaner
        )
        self.lector_metadata = LectorMetadataModelos(config_rutas.ruta_modelos)

        # Escanear configuraciones disponibles
        logger.info("Escaneando configuraciones de modelos disponibles...")
        self.configs_disponibles = self.escaner.escanear()

        # Registrar configuraciones en lectores
        for modelo, configs in self.configs_disponibles.items():
            for nombre_config, ruta_mascaras in configs:
                self.lector_mascaras.registrar_configuracion(modelo, nombre_config, ruta_mascaras)
                self.lector_metadata.registrar_ruta_json(
                    self.escaner.generar_codigo_config(modelo, nombre_config),
                    ruta_mascaras
                )

        total_configs = sum(len(c) for c in self.configs_disponibles.values())
        logger.info(f"Total configuraciones encontradas: {total_configs}")

    def integrar_fotografia(self, foto_id: str) -> Optional[DatosFotografia]:
        """
        Integra todos los datos disponibles para una fotografía.

        Args:
            foto_id: Identificador de la foto (ej: '_DSC0002')

        Returns:
            DatosFotografia consolidado o None si error crítico

        Raises:
            FileNotFoundError: Si falta característica o GT (crítico)
        """
        logger.info(f"Integrando datos de: {foto_id}")

        try:
            # 1. Cargar características (CRÍTICO)
            caracteristicas = self.lector_caracteristicas.cargar(foto_id)

            # 2. Cargar Ground Truth (CRÍTICO)
            gt = self.lector_gt.cargar(foto_id)

            # 3. Cargar máscaras de modelos (WARNING si falta)
            mascaras_modelos = {}

            for modelo, configs in self.configs_disponibles.items():
                for config, _ in configs:
                    codigo_config = self.escaner.generar_codigo_config(modelo, config)

                    mascara_data = self.lector_mascaras.cargar(codigo_config, foto_id)

                    if mascara_data is not None:
                        # Añadir metadata de modelo/config
                        mascara_data['modelo'] = modelo
                        mascara_data['config'] = config
                        mascara_data['codigo'] = codigo_config

                        mascaras_modelos[codigo_config] = mascara_data

            # 4. Cargar metadata de ejecución (WARNING si falta)
            metadata_modelos = {}

            for codigo_config in mascaras_modelos.keys():
                metadata = self.lector_metadata.cargar(codigo_config, foto_id)
                if metadata is not None:
                    metadata_modelos[codigo_config] = metadata

            # 5. Consolidar
            datos = DatosFotografia(
                foto_id=foto_id,
                ruta_imagen=self.config.ruta_imagenes / f"{foto_id}.jpg",
                ruta_json_caracteristicas=caracteristicas['ruta_json'],
                ruta_gt=gt['ruta'],
                metadatos=caracteristicas['metadatos'],
                exif_disponible=caracteristicas['exif_disponible'],
                exif=caracteristicas['exif'],
                color=caracteristicas['color'],
                saliencia=caracteristicas['saliencia'],
                calidad=caracteristicas['calidad'],
                gt_disponible=gt['existe'],
                gt_mascara=gt['mascara'],
                gt_shape=gt['shape'],
                mascaras_modelos=mascaras_modelos,
                metadata_modelos=metadata_modelos,
                timestamp_integracion=datetime.now().isoformat(),
                num_configs_disponibles=len(mascaras_modelos)
            )

            logger.info(f"  - Características: OK")
            logger.info(f"  - Ground Truth: OK (cobertura {gt['cobertura_pct']:.1f}%)")
            logger.info(f"  - Máscaras modelos: {len(mascaras_modelos)} configuraciones")
            logger.info(f"  - Metadata modelos: {len(metadata_modelos)} configuraciones")
            logger.info(f"  - EXIF disponible: {'Sí' if datos.exif_disponible else 'No'}")

            return datos

        except (FileNotFoundError, RuntimeError, ValueError) as e:
            # Errores críticos se propagan
            logger.error(f"Error crítico en {foto_id}: {e}")
            raise
        except Exception as e:
            # Otros errores se logean pero no detienen
            logger.error(f"Error inesperado en {foto_id}: {e}")
            return None

In [ ]:
# =============================================================================
# VALIDADOR DE INTEGRIDAD
# =============================================================================

class ValidadorIntegridad:
    """
    Valida integridad de datos integrados y genera estadísticas.
    """

    @staticmethod
    def validar(datos_fotos: List[DatosFotografia]) -> Dict[str, Any]:
        """
        Valida integridad de la integración de datos.

        Args:
            datos_fotos: Lista de DatosFotografia integrados

        Returns:
            Dict con estadísticas de validación y warnings/errors
        """
        if len(datos_fotos) == 0:
            return {
                'valido': False,
                'error_critico': 'No se integraron datos de ninguna fotografía'
            }

        # Estadísticas básicas
        total_fotos = len(datos_fotos)
        fotos_con_gt = sum(1 for d in datos_fotos if d.gt_disponible)
        fotos_con_exif = sum(1 for d in datos_fotos if d.exif_disponible)

        # Estadísticas de máscaras por modelo
        configs_por_foto = [d.num_configs_disponibles for d in datos_fotos]

        # Estadísticas de metadata
        metadata_por_foto = [len(d.metadata_modelos) for d in datos_fotos]

        # Contar configuraciones por modelo
        todos_codigos = set()
        for d in datos_fotos:
            todos_codigos.update(d.mascaras_modelos.keys())

        configs_por_modelo = {}
        for codigo in todos_codigos:
            modelo = codigo.split('_')[0]
            if modelo not in configs_por_modelo:
                configs_por_modelo[modelo] = []
            configs_por_modelo[modelo].append(codigo)

        # Warnings
        warnings = []
        if fotos_con_exif < total_fotos:
            warnings.append(
                f"{total_fotos - fotos_con_exif} fotos sin EXIF "
                f"({100*(total_fotos-fotos_con_exif)/total_fotos:.1f}%). "
                f"Análisis de correlaciones fotográficas limitado."
            )

        if min(configs_por_foto) < max(configs_por_foto):
            warnings.append(
                f"Número inconsistente de máscaras por foto: "
                f"min={min(configs_por_foto)}, max={max(configs_por_foto)}. "
                f"Algunas configuraciones pueden faltar en ciertas fotos."
            )

        if sum(metadata_por_foto) < sum(configs_por_foto):
            warnings.append(
                f"Metadata de ejecución no disponible para todas las configuraciones. "
                f"Total máscaras: {sum(configs_por_foto)}, Total metadata: {sum(metadata_por_foto)}"
            )

        # Resultado
        resultado = {
            'valido': True,
            'estadisticas': {
                'total_fotografias': total_fotos,
                'fotos_con_ground_truth': fotos_con_gt,
                'fotos_con_exif': fotos_con_exif,
                'porcentaje_sin_exif': 100 * (total_fotos - fotos_con_exif) / total_fotos,
                'configuraciones_por_foto': {
                    'min': min(configs_por_foto),
                    'max': max(configs_por_foto),
                    'promedio': np.mean(configs_por_foto),
                    'std': np.std(configs_por_foto)
                },
                'metadata_por_foto': {
                    'min': min(metadata_por_foto),
                    'max': max(metadata_por_foto),
                    'promedio': np.mean(metadata_por_foto),
                    'total': sum(metadata_por_foto)
                },
                'configuraciones_por_modelo': {
                    modelo: len(configs)
                    for modelo, configs in configs_por_modelo.items()
                },
                'total_configuraciones_unicas': len(todos_codigos)
            },
            'warnings': warnings
        }

        return resultado

In [ ]:
# =============================================================================
# FUNCIÓN PRINCIPAL
# =============================================================================

def ejecutar_fase1_integracion(ruta_base_tfm: str) -> Tuple[List[DatosFotografia], Dict[str, Any]]:
    """
    Ejecuta la Fase 1 completa de integración de datos.

    Args:
        ruta_base_tfm: Ruta al directorio base del proyecto TFM

    Returns:
        Tupla (lista_datos_fotografias, dict_validacion)

    Raises:
        FileNotFoundError: Si rutas críticas no existen
        RuntimeError: Si errores críticos en datos
    """
    logger.info("="*80)
    logger.info("FASE 1: INTEGRACIÓN DE DATOS")
    logger.info("="*80)

    # 1. Descubrir estructura de directorios
    logger.info("\n[1/5] Descubriendo estructura del proyecto...")
    config_rutas = ConfiguracionRutas.descubrir_desde_base(ruta_base_tfm)

    # Mostrar estructura descubierta
    config_rutas.mostrar_estructura()

    rutas_validas, errores_rutas = config_rutas.validar()
    if not rutas_validas:
        logger.error("\nErrores en la estructura de directorios:")
        for error in errores_rutas:
            logger.error(f"  - {error}")
        raise FileNotFoundError(
            "No se pudo validar la estructura del proyecto. "
            "Verifica los directorios críticos arriba."
        )

    logger.info("\n  Estructura validada correctamente")

    # 2. Obtener lista de fotografías
    logger.info("\n[2/5] Escaneando fotografías en directorio...")
    imagenes = sorted(config_rutas.ruta_imagenes.glob("*.jpg"))
    fotos_ids = [img.stem for img in imagenes]

    logger.info(f"  Fotografías encontradas: {len(fotos_ids)}")

    if len(fotos_ids) == 0:
        raise FileNotFoundError(
            f"No se encontraron imágenes JPG en {config_rutas.ruta_imagenes}"
        )

    # Mostrar primeras fotos para verificación
    if len(fotos_ids) > 0:
        logger.info(f"  Primeras fotos: {', '.join(fotos_ids[:3])}")
        if len(fotos_ids) > 3:
            logger.info(f"  ... y {len(fotos_ids) - 3} más")

    # 3. Inicializar integrador
    logger.info("\n[3/5] Inicializando integrador de datos...")
    integrador = IntegradorDatos(config_rutas)

    # 4. Integrar cada fotografía
    logger.info("\n[4/5] Integrando datos por fotografía...")
    logger.info("-"*80)

    datos_integrados = []
    errores_criticos = []

    for i, foto_id in enumerate(fotos_ids, 1):
        logger.info(f"\n[{i}/{len(fotos_ids)}] Procesando: {foto_id}")

        try:
            datos = integrador.integrar_fotografia(foto_id)

            if datos is not None:
                datos_integrados.append(datos)
            else:
                logger.warning(f"  No se pudo integrar {foto_id} (error no crítico)")

        except Exception as e:
            errores_criticos.append({
                'foto_id': foto_id,
                'error': str(e)
            })
            logger.error(f"  ERROR CRÍTICO: {e}")

    # Si hay errores críticos, detener
    if len(errores_criticos) > 0:
        logger.error("\n" + "="*80)
        logger.error("ERRORES CRÍTICOS ENCONTRADOS")
        logger.error("="*80)
        for error in errores_criticos:
            logger.error(f"  - {error['foto_id']}: {error['error']}")

        raise RuntimeError(
            f"Se encontraron {len(errores_criticos)} errores críticos. "
            f"Revisar logs arriba para detalles."
        )

    # 5. Validar integridad
    logger.info("\n[5/5] Validando integridad de datos integrados...")
    logger.info("-"*80)

    validacion = ValidadorIntegridad.validar(datos_integrados)

    if not validacion['valido']:
        raise RuntimeError(
            f"Validación de integridad falló: {validacion.get('error_critico')}"
        )

    # Mostrar estadísticas
    stats = validacion['estadisticas']
    logger.info("\nEstadísticas de integración:")
    logger.info(f"  - Total fotografías: {stats['total_fotografias']}")
    logger.info(f"  - Con Ground Truth: {stats['fotos_con_ground_truth']}")
    logger.info(f"  - Con EXIF: {stats['fotos_con_exif']} ({100-stats['porcentaje_sin_exif']:.1f}%)")
    logger.info(f"  - Configuraciones únicas: {stats['total_configuraciones_unicas']}")
    logger.info(f"  - Configs por foto (promedio): {stats['configuraciones_por_foto']['promedio']:.1f}")
    logger.info(f"  - Metadata cargada: {stats['metadata_por_foto']['total']} entradas")

    logger.info("\nConfigs por modelo:")
    for modelo, count in stats['configuraciones_por_modelo'].items():
        logger.info(f"  - {modelo}: {count} configuraciones")

    # Mostrar warnings
    if len(validacion['warnings']) > 0:
        logger.warning("\nAdvertencias:")
        for warning in validacion['warnings']:
            logger.warning(f"  - {warning}")

    # Resultado final
    logger.info("\n" + "="*80)
    logger.info("FASE 1 COMPLETADA EXITOSAMENTE")
    logger.info("="*80)
    logger.info(f"Fotografías integradas: {len(datos_integrados)}")
    logger.info(f"Máscaras totales cargadas: {sum(d.num_configs_disponibles for d in datos_integrados)}")
    logger.info(f"Metadata totales cargadas: {sum(len(d.metadata_modelos) for d in datos_integrados)}")

    return datos_integrados, validacion

In [ ]:
# =============================================================================
# PUNTO DE ENTRADA PARA TESTING
# =============================================================================

if __name__ == "__main__":

    RUTA_BASE_TFM = "/content/drive/MyDrive/TFM"

    try:
        datos_fotos, validacion = ejecutar_fase1_integracion(RUTA_BASE_TFM)

        print("\n" + "="*80)
        print("RESUMEN DE INTEGRACIÓN")
        print("="*80)
        print(f"Fotografías procesadas: {len(datos_fotos)}")
        print(f"Validación: {'EXITOSA' if validacion['valido'] else 'FALLIDA'}")

        if len(datos_fotos) > 0:
            print(f"\nEjemplo de datos integrados (primera foto):")
            ejemplo = datos_fotos[0]
            print(f"  - ID: {ejemplo.foto_id}")
            print(f"  - GT disponible: {ejemplo.gt_disponible}")
            print(f"  - EXIF disponible: {ejemplo.exif_disponible}")
            print(f"  - Máscaras cargadas: {ejemplo.num_configs_disponibles}")
            print(f"  - Metadata cargada: {len(ejemplo.metadata_modelos)}")
            print(f"  - Modelos: {list(set(m['modelo'] for m in ejemplo.mascaras_modelos.values()))}")

    except Exception as e:
        print(f"\nERROR DURANTE EJECUCIÓN: {e}")
        raise

[2025-11-11 21:12:45] INFO - ================================================================================
[2025-11-11 21:12:45] INFO - FASE 1: INTEGRACIÓN DE DATOS
[2025-11-11 21:12:45] INFO - ================================================================================
[2025-11-11 21:12:45] INFO - 
[1/5] Descubriendo estructura del proyecto...
[2025-11-11 21:12:45] INFO - Escaneando estructura en: /content/drive/MyDrive/TFM
[2025-11-11 21:12:45] INFO -   Imágenes originales encontrado en: 0_Imagenes
[2025-11-11 21:12:45] INFO -   Características fotográficas encontrado en: 1_Caracteristicas
[2025-11-11 21:12:45] INFO -   Ground Truth encontrado en: 0_Imagenes_CVAT/ground_truth_masks
[2025-11-11 21:12:45] INFO -   Resultados de modelos encontrado en: 2_Modelos
[2025-11-11 21:12:45] INFO -   Directorio de análisis: 3_Analisis
[2025-11-11 21:12:45] INFO - 
  Estructura validada correctamente
[2025-11-11 21:12:45] INFO - 
[2/5] Escaneando fotografías en directorio...



Estructura de rutas descubiertas:
  Base:             /content/drive/MyDrive/TFM
  Imágenes:         /content/drive/MyDrive/TFM/0_Imagenes
  Características:  /content/drive/MyDrive/TFM/1_Caracteristicas
  Ground Truth:     /content/drive/MyDrive/TFM/0_Imagenes_CVAT/ground_truth_masks
  Modelos:          /content/drive/MyDrive/TFM/2_Modelos
  Análisis:         /content/drive/MyDrive/TFM/3_Analisis


[2025-11-11 21:12:45] INFO -   Fotografías encontradas: 20
[2025-11-11 21:12:45] INFO -   Primeras fotos: _DSC0023, _DSC0036, _DSC0071
[2025-11-11 21:12:45] INFO -   ... y 17 más
[2025-11-11 21:12:45] INFO - 
[3/5] Inicializando integrador de datos...
[2025-11-11 21:12:45] INFO -   JSONs encontrados en subdirectorio: 1_Caracteristicas/json
[2025-11-11 21:12:45] INFO - Escaneando configuraciones de modelos disponibles...
[2025-11-11 21:13:01] INFO - Modelo mask2former: 19 configuraciones encontradas
[2025-11-11 21:13:01] INFO -   Configuraciones de mask2former:
[2025-11-11 21:13:01] INFO -     - large_ade_baja_sensibilidad_resultados: large_ade/baja_sensibilidad/resultados/mascaras
[2025-11-11 21:13:01] INFO -     - large_ade_media_sensibilidad_resultados: large_ade/media_sensibilidad/resultados/mascaras
[2025-11-11 21:13:01] INFO -     - large_ade_alta_sensibilidad_resultados: large_ade/alta_sensibilidad/resultados/mascaras
[2025-11-11 21:13:01] INFO -     - large_ade_maxima_sensibilid


RESUMEN DE INTEGRACIÓN
Fotografías procesadas: 20
Validación: EXITOSA

Ejemplo de datos integrados (primera foto):
  - ID: _DSC0023
  - GT disponible: True
  - EXIF disponible: False
  - Máscaras cargadas: 51
  - Metadata cargada: 51
  - Modelos: ['sam2', 'oneformer', 'mask2former', 'bodypix']
